In [37]:
from datetime import datetime

# Concurrencia vs Paralelismo

- Paralelismo => Caminar mientras masticas chicle
- Concurrencia => Trotas y te amarras las agujetas
- Es decir, Paralelismo es cuando podemos realizar dos cosas en el mismo espacio de tiempo sin necesidad de parar una tarea para hacer la otra
- Concurrencia es hacer dos cosas, casi al mismo tiempo, mientras se deja de hacer una se ejecuta otra para así aprovehcar el tiempo

""Con asyncio, lo que hacemos es concurrencia, es decir, en un mismo hilo ejecutamos varias cosas tratando de tomar ventaja de los momentos que una tarea hace pasusas para hacer otras, pareciendo que las hicieran al mismo tiempo""

Un ejemplo, cuando descargamos una página y esta tarda, si ejecutáramos síncronamente tendríamos que esperar a que termine de cargar para seguir trabajando pero, si lo hacemos asíncrona podríamos ejecutar otras tareas mientras se espera

# Asyncio

- Para programación asíncrona
- Es una librería para escribir código concurrente, usando **async/await**

- Este es un código síncrono, por lo que no continuará la ejecución hasta que divs1 haya finalizado.

In [48]:
def find_divisibles(inrange, div_by):
    print("finding nums in range {} divisible by {}".format(inrange, div_by))
    located = []
    for i in range(inrange):
        if i % div_by == 0:
            located.append(i)
    print("Done w/ nums in range {} divisible by {}".format(inrange, div_by))
    return located


def main():
    start = datetime.now()
    print(f'inicio: {start.second}')
    divs1 = find_divisibles(508000, 34113)
    divs2 = find_divisibles(100052, 3210)
    divs3 = find_divisibles(500, 3)
    end = datetime.now()
    print(f'fin: {end.second}')
    print(f'tiempo total: {(end - start)}')


if __name__ == '__main__':
    main()


inicio: 12
finding nums in range 508000 divisible by 34113
Done w/ nums in range 508000 divisible by 34113
finding nums in range 100052 divisible by 3210
Done w/ nums in range 100052 divisible by 3210
finding nums in range 500 divisible by 3
Done w/ nums in range 500 divisible by 3
fin: 12
tiempo total: 0:00:00.063480


- Esto nos genera un mayor tiempo de ejecución y teniendo tiempo de espera entre divs1 y divs2.
- Ahora trataremos de lidiar con ese tiempo y aprovecharlo para ejecutar otro código en ese tiempo.

- Importamos la librería asyncio de python
- Luego, nest_asyncio es una instalación (pip install nest_asyncio) para poer trabajar en jupyter notebooks

In [39]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

- Lo primero es hacer que nuestras funciones nos devuelvan una Coroutine. Esto lo hacemos envolviendo a la función con **async**
    - Agregar esta palabra no las hace Coroutines sino que indica que nos devolverá una Coroutine que podrá usarse con **await**
- Lo haremos para las funciones que querramos que se ejecuten de maner asíncrona, en ete caso las dos.
- Para poder usar *asyncio* debemos trabajar un un ciclo, el cual se crea con get_event_loop() como se muestra a continuación.

In [47]:
async def find_divisibles(inrange, div_by):
    print("finding nums in range {} divisible by {}".format(inrange, div_by))
    located = []
    for i in range(inrange):
        if i % div_by == 0:
            located.append(i)
    print("Done w/ nums in range {} divisible by {}".format(inrange, div_by))
    return located


async def main():
    start = datetime.now()
    print(f'inicio: {start.second}')
    divs1 = find_divisibles(508000, 34113)
    divs2 = find_divisibles(100052, 3210)
    divs3 = find_divisibles(500, 3)
    end = datetime.now()
    print(f'fin: {end.second}')
    print(f'tiempo total: {(end - start)}')


if __name__ == '__main__':
    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())
    except Exception as e:
        print(e)

inicio: 47
fin: 47
tiempo total: 0:00:00.000202


/usr/local/lib/python3.9/asyncio/tasks.py:256: RuntimeWarning: coroutine 'find_divisibles' was never awaited
  result = coro.send(None)


- Podemos ver que no se ejecuta cómo deseamos.
- Esto se debe a qué aún no le hemos agregado al loop.
- Para esoto, en donde hagamos llamado de la función que queremos sea asíncrona, la agregamos como tarea del loop como se muestra a continuación
- También, como ya estamos ejecutando asíncronamente la función, debemos esperar por sus resultado en algún lado. Para esto esperamos con **await asyncio.wait**, pasando como parámetro la lista de las variables que llaman a la función asíncrona.

In [41]:
async def find_divisibles(inrange, div_by):
    print("finding nums in range {} divisible by {}".format(inrange, div_by))
    located = []
    for i in range(inrange):
        if i % div_by == 0:
            located.append(i)
    print("Done w/ nums in range {} divisible by {}".format(inrange, div_by))
    return located


async def main():
    start = datetime.now()
    print(f'inicio: {start.second}')
    divs1 = loop.create_task(find_divisibles(50800000, 34113))
    divs2 = loop.create_task(find_divisibles(100052, 3210))
    divs3 = loop.create_task(find_divisibles(500, 3))
    await asyncio.wait([divs1, divs2, divs3])
    end = datetime.now()
    print(f'fin: {end.second}')
    print(f'tiempo total: {(end - start)}')


if __name__ == '__main__':
    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())
    except Exception as e:
        print(e)

inicio: 9
finding nums in range 50800000 divisible by 34113
Done w/ nums in range 50800000 divisible by 34113
finding nums in range 100052 divisible by 3210
Done w/ nums in range 100052 divisible by 3210
finding nums in range 500 divisible by 3
Done w/ nums in range 500 divisible by 3
fin: 14
tiempo total: 0:00:04.656849


- Pero vemos que se sigue ejecutando síncronamente
- Esto se debe a que la función find_divisibles sigue ejecutándose síncrnamente. No vemos un punto a que tenemos una tarea que no genera tiempo de espera o tiempos muertos como si lo fuera abrir un archivo pesado o esperar a cargar una página.
- Para ejemplificar esto mejor, modificaremos el código agregando tiempo de espera
    - para esto hacemos que verifique si el número es divisible en 50,000 para generar un poco de espera entre las llamadas y que no sea tanta espera
    - Por ejemplo, para el número más grande serán unas 2 veces que espere , para el sugundo 1 y para el tercero ninguna
    - Lo otro que hacemos es disminuir el número más grande para que solo haga 2 pausas ya que debido a las pausas el tiempo incrementará aunque seá asíncrona

In [49]:
from time import sleep


async def find_divisibles(inrange, div_by):
    print("finding nums in range {} divisible by {}".format(inrange, div_by))
    located = []
    for i in range(inrange):
        if i % div_by == 0:
            located.append(i)
        if i % 50000 == 0:
            await asyncio.sleep(0.00001)
    print("Done w/ nums in range {} divisible by {}".format(inrange, div_by))
    return located


async def main():
    start = datetime.now()
    print(f'inicio: {start.second}')
    divs1 = loop.create_task(find_divisibles(508000, 34113))
    divs2 = loop.create_task(find_divisibles(100052, 3210))
    divs3 = loop.create_task(find_divisibles(500, 3))
    await asyncio.wait([divs1, divs2, divs3])
    end = datetime.now()
    print(f'fin: {end.second}')
    print(f'tiempo total: {(end - start)}')


if __name__ == '__main__':
    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())
    except Exception as e:
        print(e)

inicio: 28
finding nums in range 508000 divisible by 34113
finding nums in range 100052 divisible by 3210
finding nums in range 500 divisible by 3
Done w/ nums in range 500 divisible by 3
Done w/ nums in range 100052 divisible by 3210
Done w/ nums in range 508000 divisible by 34113
fin: 28
tiempo total: 0:00:00.109224


- Vemos que la ejecución, con los mismos datos tarda 0.63 mientras que ahora es 0.1
- ya con el resultado podemos notar que las ejecuciones no fueron síncronas sino que primero se ejecuta la que gasta menos tiempo (la de 500). Esto se debe a que mientras las otras dos siguen ejecutándose porque buscan el múltiplo de 50,000
- Al terminar la de 500 se ejecuta la de 100052 que es la que toma menos tiempo mientras se ejecuta la primera
- Siendo la primera la que se ejecuta hasta terminar
- Todo esto nos muestra que se ejecutó de manera asíncrona

- Pero, esto nos muestra la ejecución y eso porque le colocamos el print(). No nos muestra los valores, es decir, el return de la función
- Para poder trabajar con los resultados de la función asíncrona tenemos que usar el método de la Coroutine .result() ya sean en la misma función donde enviamos al loop a la función asíncrona o retornamos la coroutine que nos retorna la función asíncrona y luego usamos .result()

In [51]:
from time import sleep


async def find_divisibles(inrange, div_by):
    print("finding nums in range {} divisible by {}".format(inrange, div_by))
    located = []
    for i in range(inrange):
        if i % div_by == 0:
            located.append(i)
        if i % 50000 == 0:
            await asyncio.sleep(0.00001)
    print("Done w/ nums in range {} divisible by {}".format(inrange, div_by))
    return located


async def main():
    start = datetime.now()
    print(f'inicio: {start.second}')
    divs1 = loop.create_task(find_divisibles(508000, 34113))
    divs2 = loop.create_task(find_divisibles(100052, 3210))
    divs3 = loop.create_task(find_divisibles(500, 3))
    await asyncio.wait([divs1, divs2, divs3])
    print(divs1.result())
    end = datetime.now()
    print(f'fin: {end.second}')
    print(f'tiempo total: {(end - start)}')
    return divs2, divs3


if __name__ == '__main__':
    try:
        loop = asyncio.get_event_loop()
        d2, d3 = loop.run_until_complete(main())
        print(d2.result())
    except Exception as e:
        print(e)

inicio: 19
finding nums in range 508000 divisible by 34113
finding nums in range 100052 divisible by 3210
finding nums in range 500 divisible by 3
Done w/ nums in range 500 divisible by 3
Done w/ nums in range 100052 divisible by 3210
Done w/ nums in range 508000 divisible by 34113
[0, 34113, 68226, 102339, 136452, 170565, 204678, 238791, 272904, 307017, 341130, 375243, 409356, 443469, 477582]
fin: 19
tiempo total: 0:00:00.098560
